In [430]:
import requests
import json
import pandas as pd
import time

# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

# Running from terminal on OSX
# export FLASK_APP=api
# export FLASK_ENV=development
# flask run

# Running with gunicorn (heroku, digitalocean, etc):
# Procfile >
# web: gunicorn wsgi --log-file -
# wsgi.py >
# from api import app as application
#
#if __name__ == "__main__":
#    application.run()

# https://devcenter.heroku.com/articles/getting-started-with-python
# https://www.digitalocean.com/community/tutorials/how-to-serve-flask-applications-with-gunicorn-and-nginx-on-ubuntu-14-04

In [431]:
# Set up heroku database > 
# heroku addons:create heroku-postgresql:hobby-dev
# Postgres db now up
# Look at config vars (should have DATABASE_URL) >
# heroku config
# Connect to remote psql > 
# heroku pg:psql
#


In [456]:
baseurl = 'https://vast-depths-89393.herokuapp.com/'
# baseurl = 'http://127.0.0.1:5000/'

In [480]:
url = baseurl +'createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [481]:
response.json()

{'signingKey_hex': '6422cff55261b8a4156ea5c336c239f6b16f785bf20ba88fd99d1241f13eaeac',
 'traderId': '8',
 'verifyKey_hex': '09e641b6850e0f0fefe221aa1d7b176a2f61fd429419e0a7c65016c3cc485910'}

In [482]:
url = baseurl + 'createMarket'

mkId = 1
# copy from response
sk = response.json()['signingKey_hex']
vk = response.json()['verifyKey_hex']
tId = response.json()['traderId']

In [483]:
content_makemarket = {"signingKey_hex": sk,
                    "traderId": tId, 
                    "verifyKey_hex": vk,
                    "marketRootId": mkId, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)

In [484]:
pd.DataFrame(response.json(), index=[0])


,checks,marketBranchId,marketMax,marketMin,marketRootId,traderId
0,False,1,1,0,1,8


In [485]:
# Create a trade
url = baseurl+'createTrade'

content_maketrade = {"signingKey_hex": sk, "traderId": tId, "verifyKey_hex": vk,
  "marketRootId": mkId, "marketBranchId": 1, "price": 0.551, "quantity":1}

In [486]:
# Post trade
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [487]:
# View json response
pd.DataFrame(response.json(), index=[0])

,checks,marketBranchId,marketRootId,price,quantity,traderId
0,"(True, True)",1,1,0.551,1,8


In [488]:
# Create a matched trade
url = baseurl+'createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [489]:
response.json()

{'checks': '(True, True)',
 'marketBranchId': 1,
 'marketRootId': 1,
 'price': 0.551,
 'quantity': -1,
 'traderId': '8'}

In [490]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 19.786114931106567 seconds.
matched trade: 1 in 12.271744966506958 seconds.
matched trade: 2 in 12.635643243789673 seconds.
matched trade: 3 in 12.783251762390137 seconds.
matched trade: 4 in 14.14153003692627 seconds.
matched trade: 5 in 14.122694969177246 seconds.
matched trade: 6 in 12.966891050338745 seconds.
matched trade: 7 in 12.589319229125977 seconds.
matched trade: 8 in 12.871303081512451 seconds.
matched trade: 9 in 12.991108179092407 seconds.


In [491]:
url = baseurl+'createMarket'

for i in range(4):
    t = time.time()

    content_makemarket = {"signingKey_hex": sk,
      "traderId": int(tId), "verifyKey_hex": vk,
      "marketRootId": i+1, "marketBranchId": 1, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

failed to create market 1
failed to create market 2
failed to create market 3
failed to create market 4


In [492]:
response.json()

{'checks': 'False',
 'marketBranchId': 1,
 'marketMax': 0.7,
 'marketMin': 0,
 'marketRootId': 4,
 'traderId': 8}

In [493]:
url = baseurl+'viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)



In [494]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,marketMin,marketMax
0,1,1,0,0.7
1,2,1,0,0.7
2,3,1,0,0.7
3,4,1,0,0.7


In [497]:
url = baseurl+'viewOrderBook'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [498]:
a = pd.read_json(response.json())
a

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [499]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = 1
    content_maketrade['price'] = 0.44
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 5.55500602722168 seconds.
matched trade: 1 in 5.545894145965576 seconds.
matched trade: 2 in 5.648215055465698 seconds.
matched trade: 3 in 5.680612087249756 seconds.
matched trade: 4 in 5.619505882263184 seconds.
matched trade: 5 in 5.576229095458984 seconds.
matched trade: 6 in 5.504082918167114 seconds.
matched trade: 7 in 5.543783903121948 seconds.
matched trade: 8 in 5.754372835159302 seconds.
matched trade: 9 in 5.803191900253296 seconds.


In [475]:
url = baseurl+'viewOpenTrades'

content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [476]:
pd.read_json(response.json())

,marketRootId,marketBranchId,price,quantity,traderId
0,1,1,0.44,10,1
1,1,1,0.44,10,2
2,1,1,0.44,10,4
3,1,1,0.44,10,5
4,1,1,0.44,10,6


In [477]:
url = baseurl+'viewMatchedTrades'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [478]:
pd.read_json(response.json())

,marketRootId,marketBranchId,price,quantity,traderId
0,1,1,0.551,0,1
1,1,1,0.551,0,2
2,1,1,0.551,0,4
3,1,1,0.551,0,5
4,1,1,0.551,0,6


In [479]:
tId

'6'